# SPARK - SPOTIFY PROJECT

**Group C**
-  Nikolas Artadi
-  Camila Vasquez
-  Assemgul Khametova
-  Miguel Frutos 

Spotify-Data 1921-2020
Audio features of songs from 1921-2020

The "data.csv" file contains more than 160.000 songs collected from Spotify Web API. The dataset is from Spotify and contains 169k songs from the year 1921 to year 2020. Each year got
top 100 songs.

**OBJECTIVE**
- Analyze the trends of songs over the course of a century.
- Build a content-based recommendation engine which suggests artists.
- Find Influential Artists in Different Genres.
- Predict popularity of song based on other features.
- Cluster different genres according to audio features.

Sources: <br>
1- https://www.kaggle.com/ektanegi/spotifydata-19212020?select=data.csv <br>
2- https://developer.spotify.com/discover/<br>
3- https://towardsdatascience.com/k-means-clustering-and-pca-to-categorize-music-by-similar-audio-features-df09c93e8b64<br>
4- https://towardsdatascience.com/8-data-science-project-ideas-from-kaggle-in-2021-83a3660e0342 <br>

Let's go for it:

# STRUCTURE/INDEX OF THE PROJECT
**1- SET UP**
- PySpark environment,
- Data source 
- Spark data abstraction (DataFrame)

**2- EXPLORE**
- Explore Dataset
- Explore Columns

**3- BUSINESS QUESTIONS**

**4- FREE STYLE & SKETCHES**

# 1. SETUP 
- PySpark environment,
- Data source 
- Spark data abstraction (DataFrame)

In [1]:
#This is specific for our course enviroment (to set up).
#If our enviroment was the one provided by Databricks, we wont need these two lines but the Azure Databricks Cluster
import findspark
findspark.init()

#importing Python packages and connecting our code through the Spark Cluster

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#This is in every spark jobs. Enables the access to Spark session.

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#First, access to the data source. Reading the CSV files calling funtions.
#Way of reading the file: InferSchema, telling spark that there is a header and where the file is located.
#As there are some commas inside the schema we should escape them with the underlying double quotes.
spotify = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("quote", "\"")\
                 .option("escape", "\"")\
                 .csv("data.csv") \
                 .cache() # optimization to make the processing faster. 
                        #Keep the processing in the cache.

# 2. EXPLORE

## A. EXPLORE DATASET

Identify the Data Types and Description of the Features

**PRIMARY KEY**:
- id (Id of track generated by Spotify)

**NUMERICAL VARIABLES**:
- **acousticness** [0-1] - Confidence measure of whether the track is acoustic.
- **danceability** [0–1] - Describes how suitable a track is for dancing based on musical attributes including tempo, rhythm, stability, beat strength, and overall regularity.
- **energy** [0–1]- Perceptual measure of intensity and activity. Energetic tracks feel fast, loud, and noisy (e.g. death metal: high energy, Bach prelude: low energy).
- **duration_ms** [Integer typically 200k to 300k]
- **instrumentalness** [0–1] - Predicts whether a track contains no vocals (values above 0.5 represent instrumental tracks whereas rap songs would have a score close to 0).
- **valence** [0–1] - Describes the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
- **popularity** [0–100] 
- **tempo** [Float typically 50 to 150] - The speed or pace of a given piece, as derived from the estimated average beat duration.
- **liveness** [0–1] - Detects the presence of an audience in the recording.
- **loudness** [Float typically -60 to 0] - The average volume across an entire track.
- **speechiness** [0–1] - Detects the presence of spoken words in a track (values above 0.66 describe tracks that are probably made entirely of spoken words, 0.33–0.66 describe tracks that may contain both music and speech, and values below 0.33 most likely represent music and other non-speech-like tracks).
- **year** [1921- 2020]

**DUMMY VARIABLES**:
- **mode**: 0 = Minor, 1 = Major
- **explicit** : 0 = No explicit content, 1 = Explicit content.

**CATEGORICAL VARIABLES**:
- **key**: All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…
- **artists**: List of artists mentioned.
- **release_date**: Date of release mostly in yyyy-mm-dd format, however precision of date may vary
- **name**: Name of the song.


### Column categorization
- **Timing related columns**: *duration_ms,year,release_date.*
- **Song Characteristics**: *accousticness, instrumentalness, tempo, loudness, speechiness, key, mode, explicit.*
- **Song Effects**: *danceability, energy, valence, popularity,liveness.*
- **Issue related columns**: *name, artists.*

### Display schema and size of the DataFrame

In [3]:
#Import additional python packages
#The last function is not so relevant for the group assignment, Raul prefers an effective code rather than fancy code lines.
#printSchema and spotify.counts are ACTIONS!!!
#".count()" Returns the number of rows.
from IPython.display import display, Markdown
spotify.printSchema()
display(Markdown("This DataFrame has **%d rows**." % spotify.count()))

root
 |-- acousticness: double (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: double (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- year: integer (nullable = true)



This DataFrame has **169909 rows**.

In [4]:
#Return the schema of this Spotify Dataframe pyspark.sql.types.StrucType
spotify.schema

StructType(List(StructField(acousticness,DoubleType,true),StructField(artists,StringType,true),StructField(danceability,DoubleType,true),StructField(duration_ms,IntegerType,true),StructField(energy,DoubleType,true),StructField(explicit,IntegerType,true),StructField(id,StringType,true),StructField(instrumentalness,DoubleType,true),StructField(key,IntegerType,true),StructField(liveness,DoubleType,true),StructField(loudness,DoubleType,true),StructField(mode,IntegerType,true),StructField(name,StringType,true),StructField(popularity,IntegerType,true),StructField(release_date,StringType,true),StructField(speechiness,DoubleType,true),StructField(tempo,DoubleType,true),StructField(valence,DoubleType,true),StructField(year,IntegerType,true)))

As there are 37 columns inside our dataframe, it is more user-friendly and visual to represent the dataset through pandas layout.

In [5]:
#"toPandas()"returns content as a Pandas Dataframe
spotify.toPandas()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


### Data Quality from the surface

Checking if there are missing/null values inside spotify dataset

In [6]:
#Using list comprehension to check if there is null values.
from pyspark.sql.functions import isnan, when, count, col

spotify.select([count(when(isnan(c), c)).alias(c) for c in spotify.columns]).toPandas()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Checking if the distinct values are the expected:
- Explicit: 2
- id: Same as the nº of rows (169909)
- key:12
- mode:2
- year: 100

In [7]:
from pyspark.sql.functions import countDistinct
print("Checking amount of distinct values in columns Year, Month, DayofMonth and DayOfWeek:")
spotify.select([countDistinct(c).alias(c) for c in spotify.columns]).toPandas()

Checking amount of distinct values in columns Year, Month, DayofMonth and DayOfWeek:


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,4714,33375,1232,50212,2332,2,169909,5401,12,1741,25313,2,132940,100,10882,1628,84548,1739,100


## B. EXPLORE COLUMNS

In [8]:
#column names of the spotify dataframe as a list
spotify.columns

['acousticness',
 'artists',
 'danceability',
 'duration_ms',
 'energy',
 'explicit',
 'id',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'name',
 'popularity',
 'release_date',
 'speechiness',
 'tempo',
 'valence',
 'year']

Statistics about the different columns

In [9]:
#".summary" specify statistics
#".select" specify columns
#".show" triggers the action
#".describe(cols)" get the basic statistics for numeric and string columns. Very similar.
spotify.select('*').summary("count", "min", "25%", "75%", "max",'mean','stddev').toPandas()

,summary,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,count,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909
1,min,0.0,"[""'In The Heights' Original Broadway Company"",...",0.0,5108,0.0,0,000G1xMMuwxNHmwVsBdtj1,0.0,0,0.0,-60.0,0,!Que Vida! - Mono Version,0,1921,0.0,0.0,0.0,1921
2,25%,0.0944,None,0.417,171040,0.263,0,None,0.0,2,0.0984,-14.47,0,24.0,12,1948.0,0.0349,93.519,0.322,1957
3,75%,0.888,None,0.667,262943,0.71,0,None,0.0868,8,0.263,-7.119,1,1980.0,48,1980.0,0.0754,135.7,0.7490000000000001,1999
4,max,0.996,['黑豹'],0.988,5403500,1.0,1,7zzuPsjj9L3M7ikqGmjN0D,1.0,11,1.0,3.855,1,화려하지 않은 고백 Confession Is Not Flashy,100,2020-06-18,0.969,244.091,1.0,2020
5,mean,0.493213976149875,None,0.5381497172015581,231406.1589733328,0.4885931303603691,0.08486307376301432,None,0.16193714313891427,5.200519101401339,0.20669034936348246,-11.370289301920401,0.7085557563166165,Infinity,31.55660971461194,1963.7319876146244,0.09405769441288801,116.94801737400672,0.5320951423473788,1977.2232312590857
6,stddev,0.37662706233783233,None,0.17534578204760803,121321.92321940271,0.26738993295713437,0.27867864925974417,None,0.30932888203837267,3.5152567984248693,0.1767964903942966,5.6667646295717695,0.45442899544073423,NaN,21.582613808557902,21.570826832323228,0.14993730259926946,30.726936654886917,0.26240763092035796,25.593167631762732


In [10]:
#Changing column datatype
#.cast (IntegerType()) -----> convert the column into integer.
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DateType
from pyspark.sql.types import BooleanType
spotify_types = spotify \
    .withColumn('acousticness',col('acousticness').cast(FloatType())) \
    .withColumn('artists',col('artists').cast(StringType())) \
    .withColumn('danceability',col('danceability').cast(FloatType())) \
    .withColumn('duration_ms',col('duration_ms').cast(IntegerType())) \
    .withColumn('energy',col('energy').cast(FloatType())) \
    .withColumn('explicit',col('explicit').cast(BooleanType())) \
    .withColumn('id',col('id').cast(StringType())) \
    .withColumn('instrumentalness',col('instrumentalness').cast(FloatType())) \
    .withColumn('key',col('key').cast(IntegerType())) \
    .withColumn('liveness',col('liveness').cast(FloatType())) \
    .withColumn('loudness',col('loudness').cast(FloatType())) \
    .withColumn('mode',col('mode').cast(BooleanType())) \
    .withColumn('name',col('name').cast(StringType())) \
    .withColumn('popularity',col('popularity').cast(IntegerType())) \
    .withColumn('release_date',col('release_date').cast(DateType())) \
    .withColumn('speechiness',col('speechiness').cast(FloatType())) \
    .withColumn('tempo',col('tempo').cast(FloatType())) \
    .withColumn('valence',col('valence').cast(FloatType())) \
    .withColumn('year',col('year').cast(IntegerType())) \

spotify_types.printSchema()

root
 |-- acousticness: float (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: float (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- release_date: date (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- year: integer (nullable = true)



# 3. BUSINESS QUESTIONS

### 1 - Create a new dataframe reflecting the evolution for "Pop/Rock/Disco/Funk" music and the different Effects the music generates on people. Cluster the evolution in quarters of a century (25 years).

Business people have provided us the Genre-Tempo (Average BPM) correlation.
- *Jazz/Reggae/Hip-Hop/Blues*: <108
- *Pop/Rock/Disco/Funk*: 108-130
- *Electro-House/Dubstep/Drum´n´n/Gabba*:>130

In [20]:
#"min(), max()" extract min and max values
#".groupBy" columns we want to group by.
#".agg" aggregations to apply to the different groups
from pyspark.sql.functions import min,max,avg

display(Markdown("**Pop/Rock/Disco/Funk EVOLUTION**"))

#Create a new column with the expected genre.
Genre_spotify = spotify\
    .withColumn("Genre", when(col("tempo")<108,"Jazz/Reggae/Hip-Hop/Blues")\
                        .when((col("tempo")>108) & (col("tempo")<=130),"Pop/Rock/Disco/Funk")\
                        .otherwise("Electro-House/Dubstep/Drum´n´n/Gabba"))\
    .cache()

#Create a new column with the CenturyQuarters.
#Filter the "Pop/Rock/Disco/Funk" genre.
#Aggregate the Effects on People.
Genre_spotify.where((col("Genre")=="Pop/Rock/Disco/Funk"))\
             .withColumn('CenturyQuarters',when(col('year')<=1946,"1921-1946")\
                                   .when(col('year')<=1971,"1946-1971")\
                                   .when(col('year')<=1996,"1971-1996")\
                                   .when(col('year')<=2021,"1996-2021")\
                                   .otherwise("unacceptable"))\
                                   .groupBy('CenturyQuarters')\
                                        .agg(avg('liveness').alias('avg_liveness'),\
                                            avg('danceability').alias('avg_danceability'),\
                                            avg('energy').alias('avg_energy'),\
                                            avg('valence').alias('avg_valence'),\
                                            avg('popularity').alias('avg_popularity'))\
        .toPandas()

**Pop/Rock/Disco/Funk EVOLUTION**

,CenturyQuarters,avg_liveness,avg_danceability,avg_energy,avg_valence,avg_popularity
0,1921-1946,0.220244,0.607480,0.289130,0.624788,1.636378
1,1946-1971,0.211458,0.555406,0.388053,0.573950,18.900031
2,1996-2021,0.192432,0.639720,0.651157,0.516799,53.986777
3,1971-1996,0.205454,0.608996,0.603036,0.609978,38.059946


In [148]:
spotify.select('*').summary("count", "min", "25%", "75%", "max",'mean','stddev').toPandas()

,summary,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,count,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909,169909
1,min,0.0,"[""'In The Heights' Original Broadway Company"",...",0.0,5108,0.0,0,000G1xMMuwxNHmwVsBdtj1,0.0,0,0.0,-60.0,0,!Que Vida! - Mono Version,0,1921,0.0,0.0,0.0,1921
2,25%,0.0944,None,0.417,171040,0.263,0,None,0.0,2,0.0984,-14.47,0,24.0,12,1948.0,0.0349,93.519,0.322,1957
3,75%,0.888,None,0.667,262943,0.71,0,None,0.0868,8,0.263,-7.119,1,1980.0,48,1980.0,0.0754,135.7,0.7490000000000001,1999
4,max,0.996,['黑豹'],0.988,5403500,1.0,1,7zzuPsjj9L3M7ikqGmjN0D,1.0,11,1.0,3.855,1,화려하지 않은 고백 Confession Is Not Flashy,100,2020-06-18,0.969,244.091,1.0,2020
5,mean,0.493213976149875,None,0.5381497172015581,231406.1589733328,0.4885931303603691,0.08486307376301432,None,0.16193714313891427,5.200519101401339,0.20669034936348246,-11.370289301920401,0.7085557563166165,Infinity,31.55660971461194,1963.7319876146244,0.09405769441288801,116.94801737400672,0.5320951423473788,1977.2232312590857
6,stddev,0.37662706233783233,None,0.17534578204760803,121321.92321940271,0.26738993295713437,0.27867864925974417,None,0.30932888203837267,3.5152567984248693,0.1767964903942966,5.6667646295717695,0.45442899544073423,NaN,21.582613808557902,21.570826832323228,0.14993730259926946,30.726936654886917,0.26240763092035796,25.593167631762732


### 2 - Spotify is exploring which are the TOP10 artists of the Century. To do so, Spotify Laboratory has created an specific formula with some weights to get the score (0/10) of a song depending of its characteristics.

|-- acousticness: Without Importance <br>
|-- danceability: **2'3 WEIGHT** <br>
|-- duration_ms: Without Importance <br>
|-- energy: **2'2 WEIGHT** <br>
|-- explicit: Without Importance <br>
|-- instrumentalness: **1'7 WEIGHT** <br>
|-- key: Without Importance <br>
|-- liveness: **1'8WEIGHT** <br>
|-- loudness: Without Importance <br>
|-- mode: Without Importance <br>
|-- popularity: Without Importance <br>
|-- speechiness: Without Importance <br>
|-- tempo: Without Importance <br>
|-- valence: **2 WEIGHT** <br>

In [180]:
#Incorporate the magic formula.
spotify_score_song = \
    spotify.withColumn('SongScore', (col('danceability')*2.3)+\
                       (col('energy')*2.2)+\
                       (col('instrumentalness')*1.7)+\
                       (col('liveness')*1.8)+\
                       (col('valence')*2))\


#Create a new column with the classification
spotify_classification = \
    spotify_score_song.withColumn("Score_Class", when(col("SongScore")>6,"Master Piece")\
                                  .when((col("SongScore")>4) & (col("SongScore")<=6),"BooM")\
                                  .otherwise("Hit"))\
#Number of songs per Artist
totalSongsArtist = \
   spotify.groupBy("Artists")\
            .agg(count(lit(1)).alias("TotalSongsArtist"))

#Total Score per Artist
totalScoreArtist = \
   spotify_score_song.groupBy("Artists").sum('SongScore').alias("TotalScore")

###We go one step further and get the top artist per category.
#MULTILABEL AGGREGATION FUNCTION: Works with the pivot method.
display(Markdown("**Top 10 artist by music effect on People per category**"))
combinedDF = \
    spotify_classification\
        .join(totalScoreArtist,"Artists")\
        .join(totalSongsArtist,"Artists")\
        .groupBy('Artists')\
        .pivot('Score_class')\
        .sum('SongScore')\
        .orderBy(col("Master Piece").desc(), col("BooM").desc(), col("Hit").desc())\
        .limit(10)\
        .show()


**Top 10 artist by music effect on People per category**

+--------------------+------------------+------------------+------------------+
|             Artists|              BooM|               Hit|      Master Piece|
+--------------------+------------------+------------------+------------------+
|['Francisco Canaro']|3766.2471738190006| 339.1181750399999| 666.6690000000002|
|['The Rolling Sto...|    1567.478463334|      351.00056298| 583.9042764999999|
|    ['The Ventures']|224.99653880000005|19.967534999999998|340.46720000000005|
|   ['Talking Heads']| 624.3107275039999|      54.090774591|     302.124889276|
|  ['The Beach Boys']|1183.7382848040002| 625.0754569390001|227.51186677999996|
| ['Michael Jackson']| 487.4030243919999|     104.402600076|219.97021203000003|
|['Bob Marley & Th...|1184.7311648060002| 98.39069450600002|     180.338792233|
|       ['New Order']|189.54113360300002|          10.83259|         177.22582|
|['Orchestra Studi...|     572.443763666|     302.936123208|          161.3982|
|    ['Depeche Mode']|     406.452319979

### 3- Raul is one of the most valuable customers inside Spotify, after several years of Sentiment Analysis we came to some conclusions about his ideal characteristics balance in music with a 95% of Confidence Level. 
### By taking into account the following characteristics, Could you recommend Raul a List with 20 songs he would dance all night long?. 
### In addition, Raul also prefers listening to music that was realease in the same season he is currently in because he beliefs that the vives and the energy are special. Could you recommend him another list?

 |-- acousticness: [0.4-1]<br>
 |-- danceability: [0.3-0.8]<br>
 |-- duration_ms: [100000-200000]<br>
 |-- energy: [0.2-0.8] <br>
 |-- explicit: 0 <br>
 |-- instrumentalness: [0.3-0.8] <br>
 |-- key: [5-10] <br>
 |-- liveness: [0.1-0.5] <br>
 |-- loudness: [(-15) - (-5)] <br>
 |-- mode: 1 <br>
 |-- popularity: [30-70] <br>
 |-- speechiness: [0.0 - 0.30]<br>
 |-- tempo: [19 - 150] <br>
 |-- valence: [0.4-0.6]<br>

In [139]:
from pyspark.sql.functions import split
from pyspark.sql.functions import month,current_date

display(Markdown("**Plug in your headphones and enjoy! This list is specifically for you Raul**"))
Raul_list = spotify\
    .where((col('acousticness')>0.4)&
           (col("acousticness")<1)&
           (col('danceability')>0.3)&\
           (col("danceability")<0.8)&\
           (col('duration_ms')>100000)&\
           (col("acousticness")<200000)&\
           (col('energy')>0.2)&\
           (col("energy")<0.8)&\
           (col("explicit")==0)&\
           (col("instrumentalness")>0.3)&\
           (col("instrumentalness")<0.8)&\
           (col("key")>5)&\
           (col("key")<10)&\
           (col("liveness")>0.1)&\
           (col("liveness")<0.5)&\
           (col("loudness")>(-15))&\
           (col("loudness")<(-5))&\
           (col("mode")==1)&\
           (col("popularity")>30)&\
           (col("popularity")<70)&\
           (col("speechiness")>0)&\
           (col("speechiness")<0.3)&\
           (col("tempo")>90)&\
           (col("tempo")<150)&\
           (col("valence")>0.4)&\
           (col("valence")<0.6))\
    .select('name','artists')\
    .orderBy('popularity')\
    .limit(20).show()

#"split" to separate between months, days, years.. and "getItem" to organize it in different columns
#"month"extracts the month from "current_date" and include it as a column
Spotify_date = spotify\
    .withColumn("split_date",split("release_date","-"or"/"))\
    .withColumn('year',col("split_date").getItem(0))\
    .withColumn('month',col("split_date").getItem(1))\
    .withColumn('day',col("split_date").getItem(2))\
    .withColumn('actual_month',month(current_date()))\
    .select('name','artists','actual_month',"release_date","day","month","year")\

display(Markdown("**Raul!, this list is designed for listening during this Season**"))

#Assign the season to the current date and the one in which the song was realeased.
#Select only the songs where the released_season is the same as the one we are in.
spotify_seasons = \
  Spotify_date \
     .withColumn('Season_release', when((col('month')>=12) | ((col('month')<=2)),"Winter")
         .when((col('month')>=3) & ((col('month')<=5)),"Spring")\
         .when((col('month')>=6) & ((col('month')<=8)),"Summer")\
         .when((col('month')>=9) & ((col('month')<=11)),"Autumn")\
         .otherwise("unacceptable"))\
     .withColumn('actual_season', when((col('actual_month')>=12) | ((col('actual_month')<=2)),"Winter")
         .when((col('actual_month')>=3) & ((col('actual_month')<=5)),"Spring")\
         .when((col('actual_month')>=6) & ((col('actual_month')<=8)),"Summer")\
         .when((col('actual_month')>=9) & ((col('actual_month')<=11)),"Autumn")\
         .otherwise("unacceptable"))\
     .where((col("actual_season"))==(col('Season_release')))\
     .select('name','artists',"release_date","month",'Season_release','actual_season')\
     .orderBy('popularity')\
     .show()


**Plug in your headphones and enjoy! This list is specificly for you Raul**

+--------------------+--------------------+
|                name|             artists|
+--------------------+--------------------+
|    What Now My Love|['Herb Alpert & T...|
|        Emily's Song| ['The Moody Blues']|
|             Godsend|  ['Beat Happening']|
|Nothing but Time ...|  ['Jackson Browne']|
|Pharaohs - Single...| ['Tears For Fears']|
|     In My Own Dream|    ['Karen Dalton']|
|Momma Miss Americ...|  ['Paul McCartney']|
|       Juke Box Mama|       ['Link Wray']|
|           All Of Me|['Oscar Peterson ...|
|            Ever New|['Beverly Glenn-C...|
|              Dreams|['The Allman Brot...|
|             Pilgrim|['Steve Earle', '...|
|       Strictly Rule|         ['Vetiver']|
|               Jingo|         ['Santana']|
|           Oh I Wept|            ['Free']|
|        Drivin' on 9|    ['The Breeders']|
|      Goin' Out West|       ['Tom Waits']|
|She's Really All ...|     ['Mac DeMarco']|
|            My Jinji|['Sunset Rollerco...|
|You Might Be Slee...|['Jakob Og

**Raul!, this list is designed for this Season**

+--------------------+--------------------+------------+-----+--------------+-------------+
|                name|             artists|release_date|month|Season_release|actual_season|
+--------------------+--------------------+------------+-----+--------------+-------------+
|Beba Mia - Instru...|['Francisco Canaro']|  1927-09-11|   09|        Autumn|       Autumn|
|Pobre Canillita -...|['Francisco Canar...|  1928-09-24|   09|        Autumn|       Autumn|
|Alfredo - Instrum...|['Francisco Canaro']|  1927-09-16|   09|        Autumn|       Autumn|
|Jinetazo - Remast...|['Francisco Canar...|  1927-09-04|   09|        Autumn|       Autumn|
|Caricias De Mujer...|['Francisco Canaro']|  1927-09-05|   09|        Autumn|       Autumn|
|Palo Verde - Inst...|['Francisco Canaro']|  1927-09-06|   09|        Autumn|       Autumn|
|Retirao - Instrum...|['Francisco Canaro']|  1927-09-04|   09|        Autumn|       Autumn|
|Noche de Reyes - ...|['Francisco Canar...|  1927-09-11|   09|        Autumn|   

# 4. FREE STYLE & SKETCHES

In [147]:
from pyspark.sql.functions import lit
print ("Most and least frequent occurrences for FlightNum, TailNum, Origin and Dest columns:")
group_artist_songs = spotify.groupBy("artists").agg(count(lit(1)).alias("artists_songs"))

mostFreqArtist   = group_artist_songs.orderBy(col("artists_songs").desc()).first()

display(Markdown("""
| %s | 
|----|
| %s | 
""" % ("mostFreqArtist", \
       "%s (%d occurrences)" % (mostFreqArtist["artists"],mostFreqArtist["artists_songs"]))))


Most and least frequent occurrences for FlightNum, TailNum, Origin and Dest columns:



| mostFreqArtist | 
|----|
| ['Эрнест Хемингуэй'] (1215 occurrences) | 


In [98]:
#"col_name[position]" using operator to access the elemnts of the array
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import split,col
from pyspark.sql import Row

spotify.withColumn("split_date",split("release_date","-"or"/") )\
        .withColumn('year',expr("split_date[0]"))\
        .withColumn('month',expr("split_date[1]"))\
        .withColumn('day',expr("split_date[2]"))\
        .select("release_date","day","month","year")\
        .show()

+------------+----+-----+----+
|release_date| day|month|year|
+------------+----+-----+----+
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-09-25|  25|   09|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-10-03|  03|   10|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-01-01|  01|   01|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-09-16|  16|   09|1928|
|        1928|null| null|1928|
|  1928-09-17|  17|   09|1928|
+------------+----+-----+----+
only showing top 20 rows



In [64]:
#Fields in a row can be accessed in this two ways.
#Way1
spotify.first().acousticness

0.995

In [118]:
#"corr(col1,col2)" returns a new column for the Pearson Correlation Coefficient
from pyspark.sql.functions import col, corr
spotify.select(corr(col('year'),col('popularity'))).show()

+----------------------+
|corr(year, popularity)|
+----------------------+
|    0.8807241644750249|
+----------------------+



In [63]:
#".when(condition,value)", equivalent to the IF function, returns one of multiple results.
#".otherwise()" equivalent to ELSE function
#"substring(col,position, number of characters)" Take the selected test from one column

from pyspark.sql.functions import substring
from pyspark.sql.functions import col, when


spotify.withColumn('CenturyQuarters',when(col('year')>1946,"1921-1946")\
                                   .when(col('year')<=1971,"1946-1971")\
                                   .when(col('year')<=1996,"1971-1996")\
                                   .when(col('year')<=2021,"1996-1921")\
                                   .otherwise("unacceptable"))\
    .select(col('CenturyQuarters')).distinct()\
    .show()    

+---------------+
|CenturyQuarters|
+---------------+
|      1921-1946|
|      1946-1971|
|      1996-1921|
|      1971-1996|
+---------------+



In [65]:
#Way2
spotify.first()['acousticness']

0.995

In [66]:
#".show(num,truncate,vertical)" returns content as a text.
spotify.show(3,5, True)

-RECORD 0-----------------
 acousticness     | 0.995 
 artists          | ['... 
 danceability     | 0.708 
 duration_ms      | 15... 
 energy           | 0.195 
 explicit         | 0     
 id               | 6K... 
 instrumentalness | 0.563 
 key              | 10    
 liveness         | 0.151 
 loudness         | -1... 
 mode             | 1     
 name             | Si... 
 popularity       | 0     
 release_date     | 1928  
 speechiness      | 0.... 
 tempo            | 11... 
 valence          | 0.779 
 year             | 1928  
-RECORD 1-----------------
 acousticness     | 0.994 
 artists          | ['... 
 danceability     | 0.379 
 duration_ms      | 28... 
 energy           | 0.... 
 explicit         | 0     
 id               | 6K... 
 instrumentalness | 0.901 
 key              | 8     
 liveness         | 0.... 
 loudness         | -2... 
 mode             | 1     
 name             | Fa... 
 popularity       | 0     
 release_date     | 1928  
 speechiness      | 0.... 
 

In [67]:
#".collect" returns contents as a list .
spotify.collect()

[Row(acousticness=0.995, artists="['Carl Woitschach']", danceability=0.708, duration_ms=158648, energy=0.195, explicit=0, id='6KbQ3uYMLKb5jDxLF7wYDD', instrumentalness=0.563, key=10, liveness=0.151, loudness=-12.428, mode=1, name='Singende Bataillone 1. Teil', popularity=0, release_date='1928', speechiness=0.0506, tempo=118.469, valence=0.779, year=1928),
 Row(acousticness=0.994, artists="['Robert Schumann', 'Vladimir Horowitz']", danceability=0.379, duration_ms=282133, energy=0.0135, explicit=0, id='6KuQTIu1KoTTkLXKrwlLPV', instrumentalness=0.901, key=8, liveness=0.0763, loudness=-28.454, mode=1, name='Fantasiestücke, Op. 111: Più tosto lento', popularity=0, release_date='1928', speechiness=0.0462, tempo=83.97200000000002, valence=0.0767, year=1928),
 Row(acousticness=0.604, artists="['Seweryn Goszczyński']", danceability=0.7490000000000001, duration_ms=104300, energy=0.22, explicit=0, id='6L63VW0PibdM1HDSBoqnoM', instrumentalness=0.0, key=5, liveness=0.119, loudness=-19.924, mode=0, 

In [68]:
#"take(num)" returns the first num of rows as a list.
spotify.take(2)

[Row(acousticness=0.995, artists="['Carl Woitschach']", danceability=0.708, duration_ms=158648, energy=0.195, explicit=0, id='6KbQ3uYMLKb5jDxLF7wYDD', instrumentalness=0.563, key=10, liveness=0.151, loudness=-12.428, mode=1, name='Singende Bataillone 1. Teil', popularity=0, release_date='1928', speechiness=0.0506, tempo=118.469, valence=0.779, year=1928),
 Row(acousticness=0.994, artists="['Robert Schumann', 'Vladimir Horowitz']", danceability=0.379, duration_ms=282133, energy=0.0135, explicit=0, id='6KuQTIu1KoTTkLXKrwlLPV', instrumentalness=0.901, key=8, liveness=0.0763, loudness=-28.454, mode=1, name='Fantasiestücke, Op. 111: Più tosto lento', popularity=0, release_date='1928', speechiness=0.0462, tempo=83.97200000000002, valence=0.0767, year=1928)]

In [69]:
#".head(num)" Returns first num of rows as a list. (SAME!)
spotify.head(2)

[Row(acousticness=0.995, artists="['Carl Woitschach']", danceability=0.708, duration_ms=158648, energy=0.195, explicit=0, id='6KbQ3uYMLKb5jDxLF7wYDD', instrumentalness=0.563, key=10, liveness=0.151, loudness=-12.428, mode=1, name='Singende Bataillone 1. Teil', popularity=0, release_date='1928', speechiness=0.0506, tempo=118.469, valence=0.779, year=1928),
 Row(acousticness=0.994, artists="['Robert Schumann', 'Vladimir Horowitz']", danceability=0.379, duration_ms=282133, energy=0.0135, explicit=0, id='6KuQTIu1KoTTkLXKrwlLPV', instrumentalness=0.901, key=8, liveness=0.0763, loudness=-28.454, mode=1, name='Fantasiestücke, Op. 111: Più tosto lento', popularity=0, release_date='1928', speechiness=0.0462, tempo=83.97200000000002, valence=0.0767, year=1928)]

In [70]:
#".first(num)" Returns first row as a list.
spotify.first()

Row(acousticness=0.995, artists="['Carl Woitschach']", danceability=0.708, duration_ms=158648, energy=0.195, explicit=0, id='6KbQ3uYMLKb5jDxLF7wYDD', instrumentalness=0.563, key=10, liveness=0.151, loudness=-12.428, mode=1, name='Singende Bataillone 1. Teil', popularity=0, release_date='1928', speechiness=0.0506, tempo=118.469, valence=0.779, year=1928)

In [71]:
#".tail(num)" Returns last num of rows as a list.
spotify.tail(1)

[Row(acousticness=0.109, artists="['Ingrid Andress']", danceability=0.512, duration_ms=214787, energy=0.428, explicit=0, id='60RFlt48hm0l4Fu0JoccOl', instrumentalness=0.0, key=0, liveness=0.105, loudness=-7.387, mode=1, name='More Hearts Than Mine', popularity=65, release_date='2020-03-27', speechiness=0.0271, tempo=80.58800000000002, valence=0.366, year=2020)]

In [72]:
#".describe(cols)" doesnt return anything special
spotify.describe()

DataFrame[summary: string, acousticness: string, artists: string, danceability: string, duration_ms: string, energy: string, explicit: string, id: string, instrumentalness: string, key: string, liveness: string, loudness: string, mode: string, name: string, popularity: string, release_date: string, speechiness: string, tempo: string, valence: string, year: string]

In [73]:
#Playing with Expresions: Transformation on one or more values in a row

#4Ways to transform the columns.

from pyspark.sql.functions import expr
spotify.select(expr('danceability+1')).first()

Row((danceability + 1)=1.708)

In [74]:
from pyspark.sql.functions import expr
spotify.select(expr('danceability')+1).first()

Row((danceability + 1)=1.708)

In [75]:
from pyspark.sql.functions import col
spotify.select(col('danceability')+1).first()

Row((danceability + 1)=1.708)

In [76]:
#Column projection: Project a set of columns

#2Ways of using it.
spotify.select('danceability').show(1)

+------------+
|danceability|
+------------+
|       0.708|
+------------+
only showing top 1 row



In [77]:
spotify.select(col('danceability')).show(1)

+------------+
|danceability|
+------------+
|       0.708|
+------------+
only showing top 1 row



In [78]:
#Displaying the first line for all the columns 
spotify.select(col('*')).show(1)

+------------+-------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|acousticness|            artists|danceability|duration_ms|energy|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|  tempo|valence|year|
+------------+-------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|       0.995|['Carl Woitschach']|       0.708|     158648| 0.195|       0|6KbQ3uYMLKb5jDxLF...|           0.563| 10|   0.151| -12.428|   1|Singende Bataillo...|         0|        1928|     0.0506|118.469|  0.779|1928|
+------------+-------------------+------------+-----------+------+--------+--------------------+----------------+---+-------

In [177]:
#"min(), max()" extract min and max values
#".groupBy" columns we want to group by.
#".agg" aggregations to apply to the different groups
from pyspark.sql.functions import min,max,avg


spotify.groupBy('year')\
        .pivot('key')\
        .agg(min('danceability').alias('min_danceability'),\
              max('danceability').alias('max_danceability'),
                avg('danceability').alias('max_danceability'))\
        .toPandas()

,year,0_min_danceability,0_max_danceability,0_max_danceability,1_min_danceability,1_max_danceability,1_max_danceability,2_min_danceability,2_max_danceability,2_max_danceability,...,8_max_danceability,9_min_danceability,9_max_danceability,9_max_danceability,10_min_danceability,10_max_danceability,10_max_danceability,11_min_danceability,11_max_danceability,11_max_danceability
0,1959,0.0764,0.889,0.520399,0.116,0.827,0.471822,0.0000,0.848,0.452603,...,0.465022,0.1520,0.873,0.493903,0.0623,0.865,0.492812,0.0814,0.851,0.494310
1,1990,0.0618,0.917,0.531802,0.125,0.895,0.586241,0.0821,0.980,0.505723,...,0.540221,0.1480,0.918,0.532600,0.0744,0.936,0.585499,0.1610,0.953,0.586055
2,1975,0.0641,0.831,0.522083,0.170,0.857,0.523790,0.0915,0.838,0.514799,...,0.535801,0.2180,0.871,0.521813,0.2470,0.870,0.559161,0.1890,0.870,0.564163
3,1977,0.1640,0.910,0.552396,0.000,0.878,0.525596,0.0648,0.903,0.520394,...,0.510386,0.1570,0.900,0.534520,0.1640,0.912,0.567196,0.0823,0.935,0.573964
4,1924,0.3460,0.778,0.557786,0.425,0.753,0.591000,0.3270,0.746,0.523737,...,0.541655,0.3940,0.760,0.633267,0.2090,0.813,0.533378,0.4710,0.742,0.604714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2002,0.1160,0.974,0.542645,0.130,0.932,0.606682,0.1030,0.964,0.576043,...,0.598924,0.0877,0.985,0.585635,0.0747,0.951,0.601983,0.1560,0.956,0.613301
96,1929,0.1880,0.853,0.647299,0.160,0.836,0.670632,0.2340,0.872,0.604810,...,0.629750,0.2510,0.838,0.677759,0.3550,0.852,0.666176,0.2870,0.790,0.671797
97,1928,0.1780,0.892,0.546533,0.196,0.869,0.502676,0.1040,0.868,0.587494,...,0.502619,0.2000,0.876,0.570595,0.1620,0.862,0.490862,0.1410,0.898,0.499338
98,1933,0.0717,0.813,0.591607,0.357,0.845,0.589810,0.2580,0.813,0.595911,...,0.610814,0.3010,0.809,0.594000,0.2490,0.829,0.620548,0.2330,0.810,0.566700


In [79]:
#".withColumn" adds new column and follows an expresion over this column
### lit() includes the literal, the selected number for the rows on that column

#When we use the action show, we are displaying text (not python object).
from pyspark.sql.functions import lit
spotify.withColumn('favoritos',lit(2)).select(col('favoritos')).show(1)

+---------+
|favoritos|
+---------+
|        2|
+---------+
only showing top 1 row



In [80]:
### upper() capital letters the selected column
from pyspark.sql.functions import upper
spotify.withColumn('favoritos',upper(col('artists'))).select(col('favoritos')).show(1)

+-------------------+
|          favoritos|
+-------------------+
|['CARL WOITSCHACH']|
+-------------------+
only showing top 1 row



In [81]:
#".drop" returns a new dataframe that drops the column specified.
spotify.drop('danceability').toPandas()

,acousticness,artists,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [82]:
#RENAMING COLUMNS (2ways)
##'.withColumnRenamed' returns a new Dataframe by renaming an existing column
spotify.withColumnRenamed('name','song').printSchema()

root
 |-- acousticness: double (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: double (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- year: integer (nullable = true)



In [83]:
#'.select(...,col(colName).alias(newName))'
spotify.select(col('name').alias('song')).show(1)

+--------------------+
|                song|
+--------------------+
|Singende Bataillo...|
+--------------------+
only showing top 1 row



In [84]:
spotify.toPandas()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [85]:
#".filter" returns the condition as expressions. Rows not satisfying the expressions are removed.
spotify.filter((col('year')>1950)&(col("key")==8)).toPandas()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9930,"['Modest Mussorgsky', 'Vladimir Horowitz']",0.343,48893,0.00569,0,1DYFmyH2msGdqbjXN3PQLw,0.797000,8,0.1320,-36.382,1,Pictures at an Exhibition: Promenade. Moderato...,0,1951,0.0419,96.856,0.466,1951
1,0.9930,['Suraiya'],0.711,175893,0.19400,0,1FEMTgmhb085sNTS9ULaLR,0.353000,8,0.0511,-10.538,1,Raaste Pe Ham Khade Hain,0,1951-01-01,0.0959,125.975,0.811,1951
2,0.9720,"['Wolfgang Amadeus Mozart', 'János Starker', '...",0.226,295240,0.10100,0,1GB2OfagSmTEDh1SfkoRkv,0.889000,8,0.2190,-20.036,1,"Cello Concerto in E-Flat Major, K. 285: II. Ro...",0,1951-10-11,0.0440,75.230,0.156,1951
3,0.8370,"['Jean Cocteau', 'André Parinaud']",0.644,278773,0.16100,0,1IYRWBduCAc8zZxpC9JFnz,0.000000,8,0.1310,-20.258,0,Le surréalisme / La poésie,0,1951,0.5880,99.224,0.428,1951
4,0.9270,"['Giuseppe Verdi', 'Mario Rossi', 'Orchestra S...",0.361,60133,0.17600,0,1McAIVgWsaqFte8P2VRjsL,0.000064,8,0.1590,-20.104,1,"Verdi : Luisa Miller : Act 1 ""Costarti, o vecc...",0,1951,0.2950,160.588,0.155,1951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8543,0.4100,"['Rauw Alejandro', 'Myke Towers', 'Sky Rompien...",0.809,184817,0.62400,1,10u2iOS0r6plfb9GknD7g4,0.000000,8,0.1080,-4.963,0,Ponte Pa' Mi,79,2020-04-16,0.1550,85.011,0.629,2020
8544,0.0274,"['Lil Tjay', 'Sheff G', 'Sleepy Hallow']",0.701,191904,0.64500,1,0w1x05XBSRIuVmKd3D0IPd,0.000174,8,0.3270,-3.084,0,Wet Em Up Pt. 2,67,2020-05-08,0.1640,107.955,0.576,2020
8545,0.2540,['YoungBoy Never Broke Again'],0.506,175339,0.78700,1,0KVEBQA98JvkmB5ZydQ13k,0.000000,8,0.1120,-6.634,1,Fine By Time,64,2020-02-21,0.1610,99.956,0.333,2020
8546,0.2100,"['LEGADO 7', 'Junior H']",0.795,218501,0.58500,0,52Cpyvd2dKb6XRn313nH87,0.000001,8,0.1120,-4.451,1,Ojos De Maniaco,68,2020-02-28,0.0374,97.479,0.934,2020


In [86]:
#".where" works in the same way as filter
#".limit" limit the results to the specified number.
spotify.where((col('year')>1950)&(col("key")==8)).limit(5).toPandas()


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.993,"['Modest Mussorgsky', 'Vladimir Horowitz']",0.343,48893,0.00569,0,1DYFmyH2msGdqbjXN3PQLw,0.797000,8,0.1320,-36.382,1,Pictures at an Exhibition: Promenade. Moderato...,0,1951,0.0419,96.856,0.466,1951
1,0.993,['Suraiya'],0.711,175893,0.19400,0,1FEMTgmhb085sNTS9ULaLR,0.353000,8,0.0511,-10.538,1,Raaste Pe Ham Khade Hain,0,1951-01-01,0.0959,125.975,0.811,1951
2,0.972,"['Wolfgang Amadeus Mozart', 'János Starker', '...",0.226,295240,0.10100,0,1GB2OfagSmTEDh1SfkoRkv,0.889000,8,0.2190,-20.036,1,"Cello Concerto in E-Flat Major, K. 285: II. Ro...",0,1951-10-11,0.0440,75.230,0.156,1951
3,0.837,"['Jean Cocteau', 'André Parinaud']",0.644,278773,0.16100,0,1IYRWBduCAc8zZxpC9JFnz,0.000000,8,0.1310,-20.258,0,Le surréalisme / La poésie,0,1951,0.5880,99.224,0.428,1951
4,0.927,"['Giuseppe Verdi', 'Mario Rossi', 'Orchestra S...",0.361,60133,0.17600,0,1McAIVgWsaqFte8P2VRjsL,0.000064,8,0.1590,-20.104,1,"Verdi : Luisa Miller : Act 1 ""Costarti, o vecc...",0,1951,0.2950,160.588,0.155,1951


In [87]:
spotify.select(col('year')).distinct().show()

+----+
|year|
+----+
|1959|
|1990|
|1975|
|1977|
|1924|
|2003|
|2007|
|2018|
|1974|
|2015|
|1927|
|1955|
|2006|
|1978|
|1925|
|1961|
|2013|
|1942|
|1939|
|1944|
+----+
only showing top 20 rows



In [88]:
#".distinct" dataframe containing the distinct rows
spotify.select(col('artists')).distinct().show()

+--------------------+
|             artists|
+--------------------+
|    ['The Drifters']|
|   ['Wanda Jackson']|
|['Elvis Presley',...|
|['Martha Reeves &...|
|['Blue Öyster Cult']|
|['Earth, Wind & F...|
|     ['Joe Jackson']|
|  ['Alain Barrière']|
|        ['Headpins']|
|['Rickie Lee Jones']|
|['Archie Shepp', ...|
|     ['Miguel Bosé']|
|['The Flaming Lips']|
|    ['Stiff Dylans']|
|['Maurice Ravel',...|
|    ['Cody Johnson']|
|          ['VÉRITÉ']|
|['Los Cadetes De ...|
|         ['Agust D']|
|    ['Hameeda Bano']|
+--------------------+
only showing top 20 rows



In [89]:
#".dropDuplicates" returns the dataframe w/o duplicates.
display(Markdown("This DataFrame has **%d rows**." % spotify.dropDuplicates().count()))

This DataFrame has **169909 rows**.

In [90]:
#".orderBy" or ".sort" returns a dataframe sorted
#sort direction can be: asc, desc, asc_nulls_first,desc_nulls_first, asc_nulls_last and desc_nulls_last
spotify.sort(col('year')).limit(5).toPandas()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.941,['Dennis Day'],0.241,196307,0.274,0,4aVy85Y2sxMwIKmAcimHp0,0.000008,0,0.0984,-9.750,0,How Can You Buy Killarny,2,1921,0.0297,90.773,0.2120,1921
1,0.996,['John McCormack'],0.518,159507,0.203,0,5uNZnElqOS3W4fRmRYPk4T,0.000000,0,0.1150,-10.589,1,The Wearing of the Green,5,1921,0.0615,66.221,0.4060,1921
2,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",5,1921,0.0366,80.954,0.0594,1921
3,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,7,1921,0.4150,60.936,0.9630,1921
4,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665,0.2530,1921


In [91]:
from pyspark.sql.functions import countDistinct
print("Checking amount of distinct values in some columns")
spotify.select([countDistinct(c).alias(c) for c in ['year','explicit','name']]).show()

Checking amount of distinct values in some columns


+----+--------+------+
|year|explicit|  name|
+----+--------+------+
| 100|       2|132940|
+----+--------+------+



In [92]:
spotify_years = spotify.withColumn('year', substring('year',1,4)).select('year').distinct().show()


+----+
|year|
+----+
|1953|
|1957|
|1987|
|1956|
|1936|
|2016|
|2012|
|2020|
|1958|
|1943|
|1972|
|1931|
|1988|
|1926|
|1938|
|2019|
|2017|
|1932|
|1977|
|1971|
+----+
only showing top 20 rows



In [93]:
from pyspark.sql.functions import length
#"length()" get the lenght of the values for a column
spotify.withColumn('year_lenght',length('year')).select('year_lenght').distinct().show()

+-----------+
|year_lenght|
+-----------+
|          4|
+-----------+



In [94]:
#"round()" Round up the given value
from pyspark.sql.functions import round
spotify.select(round('loudness').alias('round')).limit(5).show()

+-----+
|round|
+-----+
|-12.0|
|-28.0|
|-20.0|
|-15.0|
|-17.0|
+-----+



In [95]:
#"bround()" Round down the given value
from pyspark.sql.functions import bround
spotify.select(bround('loudness').alias('bround')).limit(5).show()

+------+
|bround|
+------+
| -12.0|
| -28.0|
| -20.0|
| -15.0|
| -17.0|
+------+



In [96]:
#"corr(col1,col2)" returns a new column for the Pearson Correlation Coefficient
from pyspark.sql.functions import col, corr
spotify.select(corr(col('acousticness'),col('danceability'))).show()

+--------------------------------+
|corr(acousticness, danceability)|
+--------------------------------+
|             -0.2659502509484105|
+--------------------------------+



In [97]:
#"column.getItem(position)" gets an item at position ordinal out of the array
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import split,col
from pyspark.sql import Row

spotify.withColumn("split_date",split("release_date","-"or"/") )\
        .withColumn('year',col("split_date").getItem(0))\
        .withColumn('month',col("split_date").getItem(1))\
        .withColumn('day',col("split_date").getItem(2))\
        .select("release_date","day","month","year")\
        .show()

+------------+----+-----+----+
|release_date| day|month|year|
+------------+----+-----+----+
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-09-25|  25|   09|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-10-03|  03|   10|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-01-01|  01|   01|1928|
|        1928|null| null|1928|
|        1928|null| null|1928|
|  1928-09-16|  16|   09|1928|
|        1928|null| null|1928|
|  1928-09-17|  17|   09|1928|
+------------+----+-----+----+
only showing top 20 rows



Introduce a rating 

In [100]:
spotify\
    .withColumn('CenturyQuarters', when(col('year')<=1946,"1921-1946") \
                               .when(col('year')<=1971,"1946-1971")\
                               .when(col('year')<=1996,"1971-1996")\
                               .when(col('year')<=2021,"1996-1921")\
                               .otherwise("unacceptable"))\
    .select(col('CenturyQuarters')).distinct()\
    .show()  

+---------------+
|CenturyQuarters|
+---------------+
|      1921-1946|
|      1946-1971|
|      1996-1921|
|      1971-1996|
+---------------+



How do we call a column? Well.. there are 4 different ways:

In [102]:
spotify['year']

Column<b'year'>

In [103]:
spotify.year

Column<b'year'>

In [104]:
from pyspark.sql.functions import col
col('year')

Column<b'year'>

In [105]:
from pyspark.sql.functions import column
column('year')

Column<b'year'>